In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import torch

from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [24]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets.heloc import HelocDataset
from counterfactuals.datasets.moons import MoonsDataset
from counterfactuals.datasets.law import LawDataset

from counterfactuals.optimizers.approach_three import ApproachThree

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Moons

## Train model

In [25]:
dataset = MoonsDataset(file_path="../data/origin/moons.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

In [28]:
flow = MaskedAutoregressiveFlow(features=2, hidden_features=4, context_features=1)
cf = ApproachThree(model=flow)

In [29]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=100,
    verbose=True
)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epochs:   1%|          | 1/100 [00:00<01:28,  1.12it/s]

Epoch 0, Train: 2.596069495399277, test: 1.5107005421931927


Epochs:  11%|█         | 11/100 [00:09<01:16,  1.16it/s]

Epoch 10, Train: 0.4136806981904166, test: 0.3962578194645735


...

Epochs:  81%|████████  | 81/100 [01:10<00:16,  1.14it/s]

Epoch 80, Train: 0.36569751095978215, test: 0.34506069811490864


Epochs:  91%|█████████ | 91/100 [01:19<00:07,  1.13it/s]

Epoch 90, Train: 0.36315509496313153, test: 0.36652979942468494


Epochs: 100%|██████████| 100/100 [01:27<00:00,  1.15it/s]


In [7]:
# pred_train_dataloader = DataLoader(
#     dataset=TensorDataset(torch.from_numpy(dataset.X_train), torch.from_numpy(y_pred_train)),
#     batch_size=64
# )

# pred_test_dataloader = DataLoader(
#     dataset=TensorDataset(torch.from_numpy(dataset.X_test), torch.from_numpy(y_pred_test)),
#     batch_size=64
# )

In [8]:
# # clf = LogisticRegression()
# clf = MLPClassifier((128, 64), max_iter=100)
# clf.fit(dataset.X_train, dataset.y_train)
# y_pred_train = clf.predict(dataset.X_train)
# y_pred_test = clf.predict(dataset.X_test)
# print(classification_report(dataset.y_test, y_pred_test, output_dict=False))

In [30]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       797
         1.0       1.00      1.00      1.00       842

    accuracy                           1.00      1639
   macro avg       1.00      1.00      1.00      1639
weighted avg       1.00      1.00      1.00      1639



## Generate counterfactuals

In [31]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=100, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

100it [01:01,  1.63it/s]


## Scores

In [51]:
y_cfs_pred = clf.predict(Xs_cfs)
y_clf_pred = clf.predict(dataset.X_test[:100])

np.sum(y_cfs_pred != y_clf_pred) / y_clf_pred.shape[0]

0.7

### CF Metrics for Flow predictions

In [39]:
ys_cfs_pred = cf.predict_model(Xs_cfs)
ys_orig_pred = cf.predict_model(dataset.X_test[:100])

np.sum(ys_cfs_pred != ys_orig_pred) / ys_orig_pred.shape[0]

0.76

### CF Metrics for real test data

In [40]:
ys_cfs_pred = cf.predict_model(Xs_cfs)
ys_orig_pred = dataset.y_test[:100].flatten()

np.sum(ys_cfs_pred != ys_orig_pred) / ys_orig_pred.shape[0]

0.76

# HELOC

## Train Model

In [39]:
dataset = HelocDataset(file_path="../data/origin/heloc.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [40]:
flow = MaskedAutoregressiveFlow(features=23, hidden_features=4, context_features=1)
cf = ApproachThree(model=flow)
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=100,
    verbose=True
)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epochs:   1%|          | 1/100 [00:00<00:47,  2.09it/s]

Epoch 0, Train: 36.58724835132425, test: -4.480853648746715


Epochs:  11%|█         | 11/100 [00:05<00:41,  2.14it/s]

Epoch 10, Train: -24.826751154822272, test: -24.831530739279355


...

Epochs:  81%|████████  | 81/100 [00:38<00:09,  2.08it/s]

Epoch 80, Train: -32.83665147987572, test: -32.60954150031595


Epochs:  91%|█████████ | 91/100 [00:43<00:04,  2.17it/s]

Epoch 90, Train: -33.23829447256552, test: -31.960235371309167


Epochs: 100%|██████████| 100/100 [00:47<00:00,  2.11it/s]


In [41]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       0.70      0.49      0.57       556
         1.0       0.57      0.76      0.65       490

    accuracy                           0.62      1046
   macro avg       0.63      0.62      0.61      1046
weighted avg       0.64      0.62      0.61      1046



## Generate counterfactuals

In [42]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=100, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

100it [00:46,  2.17it/s]


## Scores

In [43]:
logr = LogisticRegression()
logr.fit(dataset.X_train, dataset.y_train)
y_pred_logr = logr.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_logr, output_dict=False))

              precision    recall  f1-score   support

         0.0       0.73      0.76      0.75       556
         1.0       0.72      0.69      0.70       490

    accuracy                           0.73      1046
   macro avg       0.72      0.72      0.72      1046
weighted avg       0.73      0.73      0.73      1046



In [50]:
mlpc = MLPClassifier((128, 64), max_iter=50)
mlpc.fit(dataset.X_train, dataset.y_train)
y_pred_mlpc = mlpc.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_mlpc, output_dict=False))

              precision    recall  f1-score   support

         0.0       0.73      0.79      0.76       556
         1.0       0.73      0.66      0.70       490

    accuracy                           0.73      1046
   macro avg       0.73      0.73      0.73      1046
weighted avg       0.73      0.73      0.73      1046



/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [45]:
ys_cfs_pred = cf.predict_model(Xs_cfs)
ys_orig_pred = cf.predict_model(dataset.X_test[:100])

validity(ys_cfs_pred, ys_orig_pred)

0.65

In [46]:
y_cfs_pred_logr = logr.predict(Xs_cfs)
y_orig_pred_logr = logr.predict(dataset.X_test[:100])

validity(y_cfs_pred_logr, y_orig_pred_logr)

0.06

In [51]:
y_cfs_pred_mlpc = mlpc.predict(Xs_cfs)
y_orig_pred_mlpc = mlpc.predict(dataset.X_test[:100])

np.sum(y_cfs_pred_mlpc != y_orig_pred_mlpc) / y_orig_pred_mlpc.shape[0]

0.09

# LAW

## Train model

In [52]:
dataset = LawDataset(file_path="../data/origin/law.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

In [53]:
flow = MaskedAutoregressiveFlow(features=3, hidden_features=4, context_features=1)
cf = ApproachThree(model=flow)
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=100,
    verbose=True
)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epochs:   3%|▎         | 3/100 [00:00<00:09, 10.03it/s]

Epoch 0, Train: 9.223700612783432, test: 8.432320356369019


Epochs:  13%|█▎        | 13/100 [00:01<00:08, 10.24it/s]

Epoch 10, Train: -1.1231051646173, test: -1.1089640259742737


...

Epochs:  83%|████████▎ | 83/100 [00:08<00:01, 10.43it/s]

Epoch 80, Train: -1.2253996096551418, test: -1.1545093953609467


Epochs:  93%|█████████▎| 93/100 [00:09<00:00, 10.42it/s]

Epoch 90, Train: -1.2172787114977837, test: -1.1657378375530243


Epochs: 100%|██████████| 100/100 [00:09<00:00, 10.31it/s]


In [54]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       0.68      0.79      0.73       107
         1.0       0.77      0.66      0.71       115

    accuracy                           0.72       222
   macro avg       0.73      0.72      0.72       222
weighted avg       0.73      0.72      0.72       222



## Generate counterfactuals

In [55]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=100, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

100it [00:46,  2.15it/s]


## Score

In [56]:
logr = LogisticRegression()
logr.fit(dataset.X_train, dataset.y_train)
y_pred_logr = logr.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_logr, output_dict=False))

              precision    recall  f1-score   support

         0.0       0.68      0.79      0.73       107
         1.0       0.77      0.66      0.71       115

    accuracy                           0.72       222
   macro avg       0.73      0.72      0.72       222
weighted avg       0.73      0.72      0.72       222



In [57]:
mlpc = MLPClassifier((128, 64), max_iter=50)
mlpc.fit(dataset.X_train, dataset.y_train)
y_pred_mlpc = mlpc.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_mlpc, output_dict=False))

              precision    recall  f1-score   support

         0.0       0.68      0.80      0.74       107
         1.0       0.78      0.65      0.71       115

    accuracy                           0.73       222
   macro avg       0.73      0.73      0.72       222
weighted avg       0.73      0.73      0.72       222



In [58]:
ys_cfs_pred = cf.predict_model(Xs_cfs)
ys_orig_pred = cf.predict_model(dataset.X_test[:100])

validity(ys_cfs_pred, ys_orig_pred)

0.69

In [59]:
y_cfs_pred_logr = logr.predict(Xs_cfs)
y_orig_pred_logr = logr.predict(dataset.X_test[:100])

validity(y_cfs_pred_logr, y_orig_pred_logr)

0.68

In [60]:
y_cfs_pred_mlpc = mlpc.predict(Xs_cfs)
y_orig_pred_mlpc = mlpc.predict(dataset.X_test[:100])

np.sum(y_cfs_pred_mlpc != y_orig_pred_mlpc) / y_orig_pred_mlpc.shape[0]

0.67